# Simple Orbit notebook
In this notebook, a recorded dataset is loaded to analyze the doppler shift along the sound-source trajectory.
This code does not automatically detect the peaks and valleys corresponding to the location in the orbit.
The user should manually find these segments based on the loudness of the sound, assuming that the orbit is closest to the detector when the sound is loudest.

In [ ]:
## importing the necessary libraries
import matplotlib
#matplotlib.use('TkAgg')
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import rfft

## Load data and measurement parameters

In [ ]:
RATE = 48000  #sampling rate in Hz, check the recorder file
data = np.load("../example/raw_20230913_testrun/data_20230913_313Hz.npy")
data_filtered = data[4800:]  #to remove the first chunk of zeros in the data

taxis = np.arange(np.size(data_filtered))/RATE
plt.plot(taxis, data_filtered, '.-')  #display an overview of the data
plt.xlabel('time (s)')
plt.show()


## Inspect segments of the sound
Selecting the segment lenght and two different starting point to compare the doppler shift

In [ ]:
seglen = 5000
seg1 = 95000
seg2 = 102000
seg3 = 107000

fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

chunk1 = data_filtered[seg1:seg1+seglen]
chunk2 = data_filtered[seg2:seg2+seglen]
chunk3 = data_filtered[seg3:seg3+seglen]

line1, = ax1.plot(np.arange(seg1, seg1+seglen, dtype = int), chunk1, '.')
line2, = ax1.plot(np.arange(seg2, seg2+seglen, dtype = int), chunk2, '.')
line2, = ax1.plot(np.arange(seg3, seg3+seglen, dtype = int), chunk3, '.')
line0, = ax1.plot(data_filtered, '-', lw=0.1, alpha=0.2)

chunk1_spec = np.abs(rfft(chunk1))
chunk2_spec = np.abs(rfft(chunk2))
chunk3_spec = np.abs(rfft(chunk3))

line_fft1, = ax2.semilogx(chunk1_spec, '-', lw=2)
line_fft2, = ax2.semilogx(chunk2_spec, '-', lw=2)
line_fft3, = ax2.semilogx(chunk3_spec, '-', lw=2)

plt.show()

## Calculate the doppler shift
here, some assumptions should be made on which segment belongs to which part of the orbit. This is not as straightforward as finding the loudest sound and might need various trials and a better theoretical understanding to make a firm conclusion.

However, some estimate of the max source velocity can be made based on the shift

In [ ]:
peak1 = np.argmax(chunk1_spec)
peak2 = np.argmax(chunk2_spec)
f1 = peak1 * RATE / seglen / 2    #here we are converting the peak frequencies in samples to real SI units
f2 = peak2 * RATE / seglen / 2

print('shifted frequencies:',f1, f2)
print('doppler shift is: {} Hz'.format((f2-f1)))

## Make and estimate based on simple theory
assuming the max doppler shift is corresponding to the source coming towards and away from the recieved, we can use the formula

f2-f1 = (f2+f1)*V/c

to calculate the source velocity. Here c is the speed of sound.
The orbit radius is V/omega with omega the frequency of orbitting, here estimated to be 1.5 Hz, based on the pattern.

In [ ]:
c_sound = 340 # assumed speed of sound in m/s
V = c_sound * (f2-f1)/(f2+f1)
print ('source velocity is {} m/s'.format(V))

r_orbit = V/(2 * np.pi * 1.5)
print ('The orbit radius is {} m'.format(r_orbit))
